###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [90]:
import pandas as pd

###Archivo m8fd-ahd9.csv

Contiene información sobre incautaciones de marihuana. Incluye el número de kilos de marihuana incautada por la Fuerza Pública en el ejercicio de sus funciones. Entiéndase por Marihuana como una droga derivada de la planta del cáñamo (Cannabis), la cual es originaria de Asia (cordilleras de Himalaya). La principal sustancia psicoactiva presente en la droga es el THC o tetrahidrocannabinol y se concentra en las inflorescencias de la planta hembra. Las hojas y la planta macho contienen mucho menos THC y usualmente no se destinan directamente para su consumo, aunque si pueden venir mezcladas con las partes más activas de la planta hembra, dependiendo de la calidad del producto y su precio en el mercado. Las semillas de cannabis no contienen THC.

In [91]:
df = pd.read_csv("https://www.datos.gov.co/resource/g228-vp9d.csv?$limit=1000000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,cantidad,unidad
0,2024-07-31T00:00:00.000,5,ANTIOQUIA,05040,ANORI,0.1700,KILOGRAMO
1,2024-07-31T00:00:00.000,5,ANTIOQUIA,05148,EL CARMEN DE VIBORAL,0.0266,KILOGRAMO
2,2024-07-31T00:00:00.000,5,ANTIOQUIA,05266,ENVIGADO,0.0800,KILOGRAMO
3,2024-07-31T00:00:00.000,5,ANTIOQUIA,05001,MEDELLIN,0.5000,KILOGRAMO
4,2024-07-31T00:00:00.000,5,ANTIOQUIA,05001,MEDELLIN,2.0000,KILOGRAMO


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905409 entries, 0 to 905408
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   fecha_hecho   905409 non-null  object 
 1   cod_depto     905409 non-null  int64  
 2   departamento  905364 non-null  object 
 3   cod_muni      905409 non-null  object 
 4   municipio     905364 non-null  object 
 5   cantidad      905409 non-null  float64
 6   unidad        905409 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 48.4+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [93]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'fecha_hecho', 'cantidad', 'unidad']
confiscation_of_marijuana_MinDefensa = df[relevant_cols]
confiscation_of_marijuana_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905409 entries, 0 to 905408
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   cod_muni      905409 non-null  object 
 1   departamento  905364 non-null  object 
 2   municipio     905364 non-null  object 
 3   fecha_hecho   905409 non-null  object 
 4   cantidad      905409 non-null  float64
 5   unidad        905409 non-null  object 
dtypes: float64(1), object(5)
memory usage: 41.4+ MB


### Convertir columna fecha_hecho a tipo date

In [94]:
# Hacer una copia explícita del DataFrame
confiscation_of_marijuana_MinDefensa_ = confiscation_of_marijuana_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
confiscation_of_marijuana_MinDefensa_['fecha_hecho'] = pd.to_datetime(confiscation_of_marijuana_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
confiscation_of_marijuana_MinDefensa_['fecha_hecho'] = confiscation_of_marijuana_MinDefensa_['fecha_hecho'].dt.year

In [95]:
confiscation_of_marijuana_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010], dtype=int32)

### Verificar valores nulos

In [96]:
confiscation_of_marijuana_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,45
municipio,45
fecha_hecho,0
cantidad,0
unidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [97]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'unidad']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(confiscation_of_marijuana_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['ANTIOQUIA' 'ATLÁNTICO' 'BOGOTÁ D.C.' 'BOLÍVAR' 'CAQUETÁ' 'CAUCA' 'CESAR'
 'CÓRDOBA' 'CUNDINAMARCA' 'HUILA' 'LA GUAJIRA' 'MAGDALENA' 'META' 'NARIÑO'
 'NORTE DE SANTANDER' 'RISARALDA' 'SANTANDER' 'VALLE DEL CAUCA' 'CALDAS'
 'QUINDÍO' nan 'SUCRE' 'TOLIMA' 'VICHADA' 'ARAUCA' 'BOYACÁ' 'CHOCÓ'
 'SAN ANDRÉS ISLAS' 'PUTUMAYO' 'CASANARE' 'AMAZONAS' 'GUAVIARE' 'VAUPÉS'
 'GUAINÍA' 'AGUAS INTERNACIONALES' 'QUINDIO' 'CAQUETA' 'BOGOTA D.C.'
 'BOLIVAR' 'ARUBA' 'ATLANTICO' 'CORDOBA' 'PANAMA' 'SAN ANDRES ISLAS'
 'GUAINIA' 'BOYACA' 'COSTA RICA' 'VAUPES' 'CHOCO' 'BRASIL' 'PERU' 'CHILE'
 'ECUADOR' 'VENEZUELA' 'BOLIVIA' 'ESPAÑA' 'FRANCIA' 'REPUBLICA DOMINICANA'
 'MAR PACÍFICO' 'HONDURAS']

Categorías en la columna 'municipio':
['ANORI' 'EL CARMEN DE VIBORAL' 'ENVIGADO' ... 'ALTO BAUDO' 'PACHAVITA'
 'SATIVANORTE']

Categorías en la columna 'unidad':
['KILOGRAMO']



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [98]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [99]:
# Aplicar la función a todas las columnas categóricas
confiscation_of_marijuana_MinDefensa_[categorical_col] = confiscation_of_marijuana_MinDefensa_[categorical_col].astype(str)
for col in categorical_col:
    confiscation_of_marijuana_MinDefensa_[col] = confiscation_of_marijuana_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [100]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df confiscation_of_marijuana_MinDefensa_

In [101]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
confiscation_of_marijuana_MinDefensa_['cod_muni'] = confiscation_of_marijuana_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = confiscation_of_marijuana_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [102]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5 7 6 4]


In [103]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,905081,0.999638
1,6,199,0.000220
2,7,106,0.000117
3,4,23,0.000025


In [104]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = confiscation_of_marijuana_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    05040
1    05148
2    05266
3    05001
4    05001

Muestra de registros con longitud 6:
     cod_muni
515    760101
684    760101
7717   760101
8247   760101
8751   760101

Muestra de registros con longitud 7:
    cod_muni
127  2140101
407  5330101
439  1880101
537  5910101
687  1520101

Muestra de registros con longitud 4:
      cod_muni
5921      1111
8396      1111
8397      1111
13121     1111
13634     1111



In [105]:
# Verificar longitudes de 'cod_mupi' igual a 6 dígitos
cod_muni_6 = confiscation_of_marijuana_MinDefensa_[confiscation_of_marijuana_MinDefensa_['cod_muni'].apply(len) == 6]
cod_muni_6['departamento'].value_counts()

,count
departamento,
PANAMA,107
COSTA RICA,61
BRASIL,12
NAN,7
ECUADOR,5
CHILE,3
ARUBA,2
BOLIVIA,1
HONDURAS,1


In [106]:
# Verificar longitudes de 'cod_mupi' igual a 7 dígitos
cod_muni_7 = confiscation_of_marijuana_MinDefensa_[confiscation_of_marijuana_MinDefensa_['cod_muni'].apply(len) == 7]
cod_muni_7['departamento'].value_counts()

,count
departamento,
NAN,38
PANAMA,31
COSTA RICA,15
REPUBLICA DOMINICANA,6
ARUBA,5
CHILE,3
ECUADOR,3
VENEZUELA,2
PERU,1


In [107]:
# Verificar longitudes de 'cod_mupi' igual a 4 dígitos
cod_muni_4 = confiscation_of_marijuana_MinDefensa_[confiscation_of_marijuana_MinDefensa_['cod_muni'].apply(len) == 4]
cod_muni_4['departamento'].value_counts()

,count
departamento,
AGUAS INTERNACIONALES,17
MAR PACIFICO,6


Nota: Como se aprecia los 'cod_mun' con longitudes diferentes a 5 dígitos, que en total suman 328 registros de un total de 905.409, corresponden a incautaciones hechas en el exterior. Procedemos a excluirlas porque para efectos del proyecto no son importantes

In [108]:
confiscation_of_marijuana_MinDefensa_ = confiscation_of_marijuana_MinDefensa_[confiscation_of_marijuana_MinDefensa_['cod_muni'].apply(len) == 5]

Nota: Con el filtro anterior el 100%  de los codigos que identifican a los municipios en el df confiscation_of_marijuana_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [109]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [110]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [111]:
# Comparar listas de departamento
compare_lists(confiscation_of_marijuana_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en confiscation_of_marijuana_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en confiscation_of_marijuana_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'BRASIL', 'ESPANA', 'VENEZUELA', 'PERU', 'SAN ANDRES ISLAS'}


  Nota: Aún figuran en el df incautaciones hechas en el exterior: 'BRASIL', 'ESPANA', 'VENEZUELA', 'PERU', además el departamento SAN ANDRES ISLAS está escrito en forma diferente a su nombre real
  
  - Solucionar discrepancias en nombres de departamento SAN ANDRES ISLAS

In [112]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES ISLAS': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA'}

# Reemplazar los nombres incorrectos  el dataframe confiscation_of_marijuana_MinDefensa_
confiscation_of_marijuana_MinDefensa_['departamento'] = confiscation_of_marijuana_MinDefensa_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [113]:
# Comparar listas de departamento
compare_lists(confiscation_of_marijuana_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en confiscation_of_marijuana_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en confiscation_of_marijuana_MinDefensa_ que no están en Departamentos dept_mpios_codes:
{'VENEZUELA', 'PERU', 'BRASIL', 'ESPANA'}


In [114]:
# Verificar cuantos registros figuran a nombre de 'VENEZUELA', 'PERU', 'BRASIL', 'ESPANA'
confiscation_overseas = confiscation_of_marijuana_MinDefensa_[confiscation_of_marijuana_MinDefensa_['departamento'].isin(['VENEZUELA', 'PERU', 'BRASIL', 'ESPANA'])]
confiscation_overseas['departamento'].value_counts()

,count
departamento,
BRASIL,33
VENEZUELA,11
PERU,7
ESPANA,1


In [115]:
# Se procede a eliminar esos registros
confiscation_of_marijuana_MinDefensa_ = confiscation_of_marijuana_MinDefensa_[~(confiscation_of_marijuana_MinDefensa_['departamento'].isin(['VENEZUELA', 'PERU', 'BRASIL', 'ESPANA']))]

In [117]:
# Comparar nuevamente las listas de departamento
compare_lists(confiscation_of_marijuana_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en confiscation_of_marijuana_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en confiscation_of_marijuana_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [118]:
# Comparar listas de códigos
compare_lists(confiscation_of_marijuana_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en confiscation_of_marijuana_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en confiscation_of_marijuana_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [119]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_confiscation_of_marijuana_MinDefensa = confiscation_of_marijuana_MinDefensa_.drop(columns=columns_to_drop)

In [122]:
# Adicionar columna para trazabilidad de la fuente
final_confiscation_of_marijuana_MinDefensa['source_id'] = 28

In [123]:
final_confiscation_of_marijuana_MinDefensa.columns

Index(['cod_muni', 'fecha_hecho', 'cantidad', 'unidad', 'source_id'], dtype='object')

In [124]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'unidad': 'unit',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_confiscation_of_marijuana_MinDefensa.rename(columns=translation_map, inplace=True)

In [125]:
#Estructura final del dataset a integrar a la base de datos
final_confiscation_of_marijuana_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905029 entries, 0 to 905408
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   dane_code         905029 non-null  object 
 1   year_of_incident  905029 non-null  int32  
 2   amount            905029 non-null  float64
 3   unit              905029 non-null  object 
 4   source_id         905029 non-null  int64  
dtypes: float64(1), int32(1), int64(1), object(2)
memory usage: 38.0+ MB


## Salvar en archivo csv en el drive

In [126]:
# Guardar en archivos CSV en el drive
final_confiscation_of_marijuana_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/confiscation_of_marijuana_MinDefensa.csv', index=False)